In [ ]:
!pip install -U spacy
!python -m spacy download es_core_news_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 45.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import json

# Ruta al archivo JSON
file_path = '/content/drive/MyDrive/data.json'
print("Ya tomo la ruta")
# Leer el archivo JSON
with open(file_path, 'r') as f:
    data = json.load(f)



Ya tomo la ruta


In [ ]:
import json
import re
import nltk
import spacy

# Descargar recursos necesarios de NLTK
nltk.download('punkt')

# Cargar el modelo de spaCy para español
nlp = spacy.load("es_core_news_sm")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from json.decoder import JSONDecodeError
from datetime import datetime
import os

print(1)

def cargar_estado(todos_los_datos, state_path='state.json'):
  """Carga los valores almacenados en fichero para iniciar a partir de un estado almacenado en caso de interrupciones de red."""
  a = []
  k = 0
  m = 0
  datos = todos_los_datos
  try:
        with open(state_path, 'r', encoding='utf-8') as state_file:
          state = json.load(state_file)
          a = state['a']
          k = state['k']
          m = state['m']
          datos = state['data']
  except FileNotFoundError:
    print(f"El archivo no se encontró en {state_path}")
    if os.path.exists('/content/drive/MyDrive/state.json'):
       try:
        state_path='/content/drive/MyDrive/state.json'
        with open(state_path, 'r', encoding='utf-8') as state_file:
          state = json.load(state_file)
          a = state['a']
          k = state['k']
          m = state['m']
          datos = state['data']
       except:
         print("El archivo es corrupto")
    else:
      print("No existe fichero anterior")
  return a, k, m, datos

def guardar_estado(lista, contador, linea, datos, state_path='state.json'):
    """Guarda en un fichero el estado de la ejecucion para continuar en caso de fallas de red."""
    with open(state_path, 'w', encoding='utf-8') as state_file:
      state = {'a': lista, 'k': contador, 'data': datos, 'm': linea}
      json.dump(state, state_file, ensure_ascii=False)
    !cp state.json /content/drive/MyDrive
def extraer_entidades_nltk(texto,bool=False):
    geo_entities=[]
    if bool:
        palabras_capitalizadas = [palabra.title() for palabra in texto.split()]
        texto = ' '.join(palabras_capitalizadas)
    doc = nlp(texto)
    geo_entities.extend([ent.text for ent in doc.ents if ent.label_ == "LOC"])
    return geo_entities

def patron_re(entidades_nltk):
    enties_re = '|'.join(re.escape(ent) for ent in entidades_nltk)
    patron = re.compile(rf'\b(\d+ y\s*\d+ (?:{enties_re})|(?:{enties_re}),\s*\d+\s*y\s*\d+|bloque \d+ en \b[A-Za-z]+\b|bloque \d+ de \b[A-Za-z]+\b|\d+\w*\s*en (?:{enties_re})|\d+\w*\s*de (?:{enties_re})|\d+\w*\s+entre\s+\d+\w*\s+y\s+\d+\w*)\b', re.IGNORECASE)
    return patron

def extraer_entidades_geograficas(texto, patron):
    coincidencias = patron.findall(texto)
    return coincidencias
    #for coin in coincidencias:
    #    print("Entidad geográfica encontrada:", coin)
#a=[]
#k=0
a, k, m, todos_los_datos = cargar_estado(data)
print(1)
def sumar_longitudes_json(todos_los_datos, output_file,a,k,m):
    start = datetime.now()
    m_inicial = m
    print(0)
    data = todos_los_datos
    for index, (key, value) in enumerate(data.items()):
      if index + 1 <= m:
        continue
      else:

        end = datetime.now()
        total_m = len(data.keys()) - m_inicial
        duracion = end - start
        procesados = m - m_inicial
        estimado = (total_m * duracion) / procesados if procesados > 0 else 0

        m += 1
        print(f'Procesando {m} de {len(data.keys())} mensajes. Tiempo estimado:', estimado)
        if len(value) >= 3 and isinstance(value[2], list):
            for i, items in enumerate(value[2]):
                k+=1
                print('\t', k, f' ({i+1} de {len(value[2])} comentarios)')
                patron = re.compile(r'Habana vieja|Regla|San Miguel del Padrón|San Miguel|Guanabacoa|gbcoa|San Miguel del Padron|Playa|Marianao|Lisa|Habana del Este|Centro Habana|Cerro|Diez de Octubre|10 de Octubre|Plaza de la Revolución|Plaza de la Revolucion|Plaza de la Rebolución|Plaza de la Rebolucion|Plaza|Arroyo Naranjo|Boyeros|Boyero|Cotorro', re.IGNORECASE)
                coincidencias = patron.findall(items[1])
                #print(coincidencias)
                if len(coincidencias):
                    a.append([coincidencias,items[1]])
                    items.append(coincidencias)
                else:
                    entidades_nltk = extraer_entidades_nltk(items[1])
                    #entidades_nltk_true = extraer_entidades_nltk(items[1],True)
                    patron = patron_re(entidades_nltk)
                    entidades=extraer_entidades_geograficas(items[1], patron)
                    if len(entidades):
                        a.append([entidades,items[1]])
                        items.append(entidades)
                    else:
                        a.append([entidades_nltk,items[1]])
        guardar_estado(a, k, m, data)

    with open(output_file, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

sumar_longitudes_json(data, 'data_enties.json',a,k,m)
for i in a :
    print(i)


_

Streaming output truncated to the last 5000 lines.
[['Llevamos'], 'Ya encontraron o están reparando la avería de lawton??? Llevamos más de 4 horas ya']
[[], 'Hasta cuando es el apagón!!!!! Y amas de 4 horas!!!!!!!']
[['Hola'], 'Hola, hasta cuando es el apagón en reparto Abel Santamaría,cuál es su causa']
[['Estancia', 'san pedro'], 'Estancia entre san pedro y lombillo en nuevo vedado sin corriente']
[[], '@AsistenteEnLineaEELH_2 nos pueden explicar esta situación de quitar y poner la corriente y a esta hora sin además previo aviso, día entre semana que los muchachos tienen que ir a la escuela y los padres a trabajar , con este calor insoportable, de verdad que el tema UNE no deja de ser tema de que hablar']
[['plaza'], 'Hasta cuando en plaza 11 y 12 vedado tengan consideración y respeto por favor el calor es horrible']
[['centro Habana'], 'Hola que pasa en centro Habana por favor asta cuándo es esto']
[['Lawton'], 'Hasta que hora va a ser lo del puente partido ya llevamos 4 horas sin l

''

In [ ]:
!cp data_entities.json /content/drive/MyDrive